The Boston App

In [63]:
! pip install haversine

Defaulting to user installation because normal site-packages is not writeable


In [ ]:

! pip install --upgrade openai

In [7]:


import requests

url = 'https://data.boston.gov/api/3/action/package_list'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Convert JSON response to a Python dictionary
else:
    print("Failed to retrieve data")
    data = {}

def fetch_datasets(dataset_ids):
    with open('datasets_info.txt', 'w') as file:
        for dataset_id in dataset_ids:
            url = f'https://data.boston.gov/api/3/action/package_show?id={dataset_id}'
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                file.write(str(data) + '\n\n')
            else:
                file.write(f'Failed to retrieve data for dataset ID {dataset_id}\n\n')

# Fetch datasets and write to file
#fetch_datasets(data['result'])



In [8]:
import requests

def get_dataset_url(resource_id):
    base_url = "https://data.boston.gov/api/3/action/package_show"
    params = {'id': resource_id}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        # Assuming the first resource in the package is the one we want
        return data['result']['resources'][0]['url']
    else:
        print("Failed to get data:", response.status_code)
        return None


In [9]:
import pandas as pd

def load_dataset_into_dataframe(url):
    if url:
        return pd.read_csv(url)
    else:
        print("Invalid URL")
        return None


In [10]:
import pandas as pd
import requests

# List of dataset name IDs
datasets = {
    'approved-building-permits': 'Approved Building Permits',
    'food-establishment-inspections': 'Food Establishment Inspections',
    'street-sweeping-schedules': 'Street Sweeping Schedules',
    'big-belly-locations': 'Big Belly Trash Receptacles',
    'trash-collection-days': 'Trash Collection Days',
    'boston-street-segments': 'Boston Street Segments',
    'city-council-roll-call-votes': 'City Council Roll Call Votes',
    'moving-truck-permits': 'Moving Truck Permits'
}


dfs = {}

# Base URL for the API
api_base_url = 'https://data.boston.gov/api/3/action/package_show?id='

for dataset_id, dataset_name in datasets.items():
    response = requests.get(api_base_url + dataset_id)
    if response.status_code == 200:
        print(f'reading {dataset_name}')
        package = response.json()['result']

        # Filter for CSV resources
        csv_resources = [resource for resource in package['resources'] if resource['format'].lower() == 'csv']

        if csv_resources:
            # Assuming the most recent CSV resource is the first in the list
            most_recent_csv_resource = csv_resources[0]
            data_url = most_recent_csv_resource['url']
            df = pd.read_csv(data_url, on_bad_lines='warn')
            dfs[dataset_id] = df
            print(f'Dataframe for {dataset_name} (CSV) created.')
        else:
            print(f'No CSV resources found for {dataset_name}.')
    else:
        print(f'Failed to fetch data for {dataset_name}.')


reading Approved Building Permits


/tmp/ipykernel_25770/2775520338.py:35: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url, on_bad_lines='warn')


Dataframe for Approved Building Permits (CSV) created.
reading Food Establishment Inspections
Dataframe for Food Establishment Inspections (CSV) created.
reading Street Sweeping Schedules
Dataframe for Street Sweeping Schedules (CSV) created.
reading Big Belly Trash Receptacles
Dataframe for Big Belly Trash Receptacles (CSV) created.
reading Trash Collection Days
Dataframe for Trash Collection Days (CSV) created.
reading Boston Street Segments
Dataframe for Boston Street Segments (CSV) created.
reading City Council Roll Call Votes
Dataframe for City Council Roll Call Votes (CSV) created.
reading Moving Truck Permits
Dataframe for Moving Truck Permits (CSV) created.


In [11]:
def extract_columns(dataframes_dict):
    columns_dict = {}
    for dataset_id, df in dataframes_dict.items():
        # Extracting column names from each dataframe and storing them as a list
        columns_dict[dataset_id] = df.columns.tolist()
    return columns_dict

In [12]:
columns_dict = extract_columns(dfs)

In [13]:
print(columns_dict)

{'approved-building-permits': ['object_id', 'permitnumber', 'worktype', 'permittypedescr', 'description', 'comments', 'applicant', 'declared_valuation', 'total_fees', 'issued_date', 'expiration_date', 'status', 'owner', 'occupancytype', 'sq_feet', 'address', 'city', 'state', 'zip', 'property_id', 'parcel_id', 'gpsy', 'gpsx', 'geom_2249', 'lat', 'long', 'geom_4326'], 'food-establishment-inspections': ['businessname', 'dbaname', 'legalowner', 'namelast', 'namefirst', 'licenseno', 'issdttm', 'expdttm', 'licstatus', 'licensecat', 'descript', 'result', 'resultdttm', 'violation', 'viollevel', 'violdesc', 'violdttm', 'violstatus', 'statusdate', 'comments', 'address', 'city', 'state', 'zip', 'property_id', 'location'], 'street-sweeping-schedules': ['main_id', 'st_name', 'dist', 'dist_name', 'start_time', 'end_time', 'side', 'from', 'to', 'miles', 'section', 'one_way', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',

In [14]:
def print_location_data_samples(dfs):
    # Define the location-related columns for each dataset
    location_columns = {
        'approved-building-permits': ['address', 'city', 'state', 'zip', 'gpsy', 'gpsx', 'geom_2249', 'lat', 'long', 'geom_4326'],
        'food-establishment-inspections': ['address', 'city', 'state', 'zip', 'location'],
        'street-sweeping-schedules': ['st_name', 'from', 'to'],
        'big-belly-locations': ['Location'],
        'moving-truck-permits': ['city', 'state', 'zip', 'lat', 'long', 'geom_4326'],
        # Add other datasets if needed
    }

    for dataset_id, df in dfs.items():
        print(f"\nSample data from dataset: {dataset_id}")

        # Check if the dataset has defined location columns
        if dataset_id in location_columns:
            for col in location_columns[dataset_id]:
                if col in df.columns:
                    # Print a sample from the column
                    print(f"Sample from {col}:")
                    print(df[col].dropna().head(5))
                else:
                    print(f"Column {col} not found in {dataset_id}")
        else:
            print(f"No location data columns defined for {dataset_id}")

# Example usage
# Assume 'dfs' is your dictionary containing dataset IDs as keys and DataFrames as values
print_location_data_samples(dfs)



Sample data from dataset: approved-building-permits
Sample from address:
0          181-183 State ST
1         175 W Boundary RD
2            15 Prospect ST
3      211 W Springfield ST
4    14 William Jackson AVE
Name: address, dtype: object
Sample from city:
0          Boston
1    West Roxbury
2     Charlestown
3         Roxbury
4        Brighton
Name: city, dtype: object
Sample from state:


0    MA
1    MA
2    MA
3    MA
4    MA
Name: state, dtype: object
Sample from zip:
0    2109.0
1    2132.0
2    2129.0
3    2118.0
4    2135.0
Name: zip, dtype: object
Sample from gpsy:
0    2.956235e+06
1    2.920239e+06
2    2.962078e+06
3    2.949423e+06
4    2.950791e+06
Name: gpsy, dtype: float64
Sample from gpsx:
0    777000.467775
1    751016.119559
2    775710.380542
3    769648.312793
4    749690.298790
Name: gpsx, dtype: float64
Sample from geom_2249:
0    0101000020C9080000014080EF50B6274128B89653E58D...
1    0101000020C908000081DB363D50EB264164AA649F9747...
2    0101000020C90800007E6BD6C23CAC2741422F500F4F99...
3    0101000020C9080000025726A0E07C274183505E499780...
4    0101000020C9080000FCFDFA98F4E02641F6694F594383...
Name: geom_2249, dtype: object
Sample from lat:
0    42.359190
1    42.260750
2    42.375243
3    42.340600
4    42.344600
Name: lat, dtype: float64
Sample from long:
0   -71.052924
1   -71.149611
2   -71.057585
3   -71.080251
4   -71.154051
Name: long, dtyp

In [15]:
def access_approved_building_permits(df):
    # Assuming the DataFrame already has 'lat' and 'long' columns
    return df

def access_location_food_establishment_inspections(df):
    # Split the 'location' column into 'lat' and 'long'
    df[['lat', 'long']] = df['location'].str.extract(r'\((.*), (.*)\)')
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['long'] = pd.to_numeric(df['long'], errors='coerce')
    return df

def access_location_big_belly_locations(df):
    # Split the 'Location' column into 'lat' and 'long'
    df[['lat', 'long']] = df['Location'].str.extract(r'\((.*), (.*)\)')
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['long'] = pd.to_numeric(df['long'], errors='coerce')
    return df

def access_moving_truck_permits_location(df):
    # Assuming the DataFrame already has 'lat' and 'long' columns
    return df

# Example usage
# Assuming dfs is your dictionary with dataset IDs as keys and DataFrames as values

dfs['moving-truck-permits'] = access_moving_truck_permits_location(dfs['moving-truck-permits'])
dfs['approved-building-permits'] = access_approved_building_permits(dfs['approved-building-permits'])
dfs['food-establishment-inspections'] = access_location_food_establishment_inspections(dfs['food-establishment-inspections'])
dfs['big-belly-locations'] = access_location_big_belly_locations(dfs['big-belly-locations'])


In [43]:
def access_date_approved_building_permits(df):
    # Convert 'issued_date' to datetime. If timezone-naive, localize to UTC
    df['date'] = pd.to_datetime(df['issued_date'], errors='coerce')
    if df['date'].dt.tz is None:
        df['date'] = df['date'].dt.tz_localize('UTC')
    return df

def access_date_food_establishment_inspections(df):
    # Convert 'issdttm' to datetime. If timezone-naive, localize to UTC
    # for each row check all columns ending in dttm and convert to datetime, then pick the latest one and rename it to date
    date_columns = [col for col in df.columns if col.endswith('dttm')]
    df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce')
    df['date'] = df[date_columns].max(axis=1)
    df = df.drop(columns=date_columns)
    if df['date'].dt.tz is None:
        df['date'] = df['date'].dt.tz_localize('UTC')
    
    return df

def access_date_big_belly_locations(df):
    # Assuming this dataset does not have a date column
    return df

def access_date_moving_truck_permits(df):
    # Standardize 'issued_date' to datetime
    df['issued_date'] = pd.to_datetime(df['issued_date'], errors='coerce')
    if df['issued_date'].dt.tz is None:
        df['issued_date'] = df['issued_date'].dt.tz_localize('UTC')
    # Standardize 'expiration_date' to datetime
    df['expiration_date'] = pd.to_datetime(df['expiration_date'], errors='coerce')
    if df['expiration_date'].dt.tz is None:
        df['expiration_date'] = df['expiration_date'].dt.tz_localize('UTC')

    return df

# Example usage
dfs['moving-truck-permits'] = access_date_moving_truck_permits(dfs['moving-truck-permits'])
dfs['approved-building-permits'] = access_date_approved_building_permits(dfs['approved-building-permits'])
dfs['food-establishment-inspections'] = access_date_food_establishment_inspections(dfs['food-establishment-inspections'])

In [53]:
import pandas as pd
from haversine import haversine

def filter_datasets_by_location(dfs, lat, lon, radius, dataset_ids):
    def is_within_radius(lat1, lon1, lat2, lon2, radius):
        return haversine((lat1, lon1), (lat2, lon2)) <= radius

    filtered_dfs = {}
    for dataset_id in dataset_ids:
        if dataset_id in dfs:
            df = dfs[dataset_id]
            # Assuming columns 'lat' and 'long' exist and are correctly formatted
            if 'lat' in df.columns and 'long' in df.columns:
                # Filter the DataFrame
                mask = df.apply(lambda row: is_within_radius(lat, lon, row['lat'], row['long'], radius), axis=1)
                filtered_dfs[dataset_id] = df[mask]
            else:
                print(f"Latitude/Longitude columns not found in {dataset_id}")

    return filtered_dfs

# Usage Example
# dfs: dictionary with dataset IDs as keys and DataFrames as values
# lat, lon: your reference latitude and longitude
# radius: radius in kilometers
# dataset_ids: list of dataset IDs to filter
# result = filter_datasets_by_location(dfs, lat, lon, radius, dataset_ids)

#Hilton Boston Park Plaza = 42.3495242,-71.0697654
dataset_id_array = ['approved-building-permits','food-establishment-inspections','big-belly-locations','moving-truck-permits']
location_filtered_result = filter_datasets_by_location(dfs, 42.345197, -71.077664, .5, dataset_id_array)



In [54]:
def filter_datasets_by_date(dfs, start_date, end_date):
    filtered_dfs = {}
    for dataset_id, df in dfs.items():
        print(f'filtering {dataset_id}')
        if 'date' in df.columns:
            mask = (df['date'] >= start_date) & (df['date'] <= end_date)
        elif 'issued_date' in df.columns and 'expiration_date' in df.columns:
            mask = ((df['issued_date'] >= start_date) & (df['issued_date'] <= end_date)) | \
                   ((df['expiration_date'] >= start_date) & (df['expiration_date'] <= end_date))
        elif 'issued_date' in df.columns:
            mask = (df['issued_date'] >= start_date) & (df['issued_date'] <= end_date)
        elif 'expiration_date' in df.columns:
            mask = (df['expiration_date'] >= start_date) & (df['expiration_date'] <= end_date)
        else:
            print(f"No suitable date column found in {dataset_id}, returning original DataFrame.")
            filtered_dfs[dataset_id] = df
            continue

        filtered_dfs[dataset_id] = df.loc[mask]
    return filtered_dfs

# Example usage
# start_date and end_date should be pandas Timestamp objects
# filtered_dfs = filter_datasets_by_date(dfs, start_date, end_date)


# Example usage
# start_date and end_date should be pandas Timestamp objects
# filtered_dfs = filter_datasets_by_date(dfs, start_date, end_date)


# Example usage
# Assuming dfs contains datasets processed by both location and date access functions
start_date = pd.to_datetime('2023-11-20', utc=True)
end_date = pd.to_datetime('2023-11-30', utc=True)
date_filtered_result = filter_datasets_by_date(location_filtered_result, start_date, end_date)


filtering approved-building-permits
filtering food-establishment-inspections
filtering big-belly-locations
No suitable date column found in big-belly-locations, returning original DataFrame.
filtering moving-truck-permits


In [55]:
def save_dfs_to_csv(dfs, folder_path='output/'):
    import os

    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)

    for dataset_id, df in dfs.items():
        file_name = f"{folder_path}{dataset_id}.csv"
        df.to_csv(file_name, index=False)
        print(f"Saved {dataset_id} to {file_name}")

# Usage
# Assuming 'dfs' is your dictionary containing the processed data
save_dfs_to_csv(date_filtered_result)


Saved approved-building-permits to output/approved-building-permits.csv
Saved food-establishment-inspections to output/food-establishment-inspections.csv
Saved big-belly-locations to output/big-belly-locations.csv
Saved moving-truck-permits to output/moving-truck-permits.csv


In [56]:
import pandas as pd

def consolidate_csvs_to_text(csv_files, output_file):
    with open(output_file, 'w') as outfile:
        for dataset_id, csv_file in csv_files.items():
            try:
                df = pd.read_csv(csv_file)
                outfile.write(f"Dataset ID: {dataset_id}\n")
                df.to_string(outfile, header=True, index=False)
                outfile.write("\n\n")  # Adding a couple of new lines for separation
            except Exception as e:
                outfile.write(f"Failed to process {csv_file}: {e}\n\n")


In [57]:
# Example usage
csv_files = {
    'approved-building-permits': './output/approved-building-permits.csv',
    'food-establishment-inspections': './output/food-establishment-inspections.csv',
    'big-belly-locations': './output/big-belly-locations.csv',
    'moving-truck-permits' : './output/moving-truck-permits.csv'
}
consolidate_csvs_to_text(csv_files, 'output/consolidated_data.txt')

# Give the data to the Boston GPT for analysis

# Let's try another location: Roxbury YMCA

In [58]:
#Roxbury YMCA coordinates 42.318389,-71.0849831

#YMCA_location_filtered_result = filter_datasets_by_location(dfs, 42.318389, -71.0849831, .25, dataset_id_array)
# Example usage
# Assuming dfs contains datasets processed by both location and date access functions
#start_date = pd.to_datetime('2023-11-20', utc=True)
#end_date = pd.to_datetime('2023-11-30', utc=True)
#YMCA_date_filtered_result = filter_datasets_by_date(YMCA_location_filtered_result, start_date, end_date)
#save_dfs_to_csv(YMCA_date_filtered_result)

# Example usage
csv_files = {
    'approved-building-permits': './output/approved-building-permits.csv',
    'food-establishment-inspections': './output/food-establishment-inspections.csv',
    'big-belly-locations': './output/big-belly-locations.csv',
    'moving-truck-permits' : './output/moving-truck-permits.csv'
}
#consolidate_csvs_to_text(csv_files, 'output/ymca_consolidated_data.txt')

In [60]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [62]:
from openai import OpenAI
client = OpenAI()

prompt = "The following is a list of datasets with records pertinent to a central location. Provide a human readable report in english of the activities, with a focus on those that might affect their day to day life\n"

with open('output/consolidated_data.txt', 'r') as file:
    prompt += file.read()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="The approved-building-permits dataset includes records of approved electrical permits. Some of the activities include electrical work at La Padrona restaurant, installation of wiring at Northeastern University, remodeling of a kitchen, renovation of a space at Flare Capital, installation of a solar PV system, restoration of power due to a fire, and kitchen renovations. These activities may affect people's day-to-day lives by impacting the availability and quality of electrical services.\n\nThe big-belly-locations dataset includes geographic coordinates of various locations in Boston where Big Belly waste and recycling containers are located. This may be useful to residents and visitors looking for waste disposal options in the area.\n\nLastly, the moving-truck-permits dataset contains records of street occupancy permits for moving trucks in different areas. This might affect traffic and parking availability in the listed locations, potentially impacting l

In [64]:
def save_response_to_file(response, filename):
    with open(filename, 'w') as file:
        file.write(response)

# Example usage
response_content = completion.choices[0].message.content  # Assuming completion is your API response
save_response_to_file(response_content, 'gpt3_English_response_output.txt')


In [65]:
#output the response in human readable format
from pprint import pprint

pprint(response_content)


('The approved-building-permits dataset includes records of approved '
 'electrical permits. Some of the activities include electrical work at La '
 'Padrona restaurant, installation of wiring at Northeastern University, '
 'remodeling of a kitchen, renovation of a space at Flare Capital, '
 'installation of a solar PV system, restoration of power due to a fire, and '
 "kitchen renovations. These activities may affect people's day-to-day lives "
 'by impacting the availability and quality of electrical services.\n'
 '\n'
 'The big-belly-locations dataset includes geographic coordinates of various '
 'locations in Boston where Big Belly waste and recycling containers are '
 'located. This may be useful to residents and visitors looking for waste '
 'disposal options in the area.\n'
 '\n'
 'Lastly, the moving-truck-permits dataset contains records of street '
 'occupancy permits for moving trucks in different areas. This might affect '
 'traffic and parking availability in the listed loc